## Riksantikvarieämbetet RAÄ - kulturarvsdata - UGC
version 1.3 

get UGC data [see API](http://ugc.kulturarvsdata.se/UGC-hub/api/documentation)
and understand how they link Europeana etc... 

* this [notebook](https://github.com/salgo60/open-data-examples/blob/master/Riksantikvarie%C3%A4mbetet%20RA%C3%84%20UGC.ipynb)
* [blogpost](https://minancestry.blogspot.com/2020/06/check-status-kulturnav-raa.html)

In [2]:
# get all records UGC just sends 200 records --> we need to loop and I guess max3000000 records
import urllib3, json
import pandas as pd  
from tqdm.notebook import trange 
http = urllib3.PoolManager()
maxCount=1000
urlbase = "http://ugc.kulturarvsdata.se/UGC-hub/api?method=retrieve&scope=all&maxCount=" + str(maxCount) + "&objectUri=all&format=json&selectFrom="
pd.set_option("display.max.columns", None) 
dftot = pd.DataFrame()
maxCount=1000
for i in trange(1,3000000,maxCount):
    url = urlbase + str(i)
    r = http.request('GET', url)
    data = json.loads(r.data)
    #print(data["response"])
    dftot = dftot.append(pd.DataFrame(data["response"]["relations"]),sort=False)
print(dftot.head())
print(dftot.userName.unique()) 


  0%|          | 0/3000 [00:00<?, ?it/s]

KeyError: 'relations'

In [ ]:
print(dftot.info())

In [ ]:
print(dftot.applicationName.unique())

In [ ]:
dftot.relationType.unique()

### UGC  
11 relationstyper jmf Wikidata > [7650 egenskaper](https://w.wiki/DYS) dvs. om målet är att "användare kan bidra själv med semantiska länkar" så känns det bättre om användaren gör det hos Wikidata där det finns stöd att även ange [semantiska länkar i bilderna](https://www.youtube.com/watch?v=lmWmMIuCJVM&t=158s) ?!?!? 

In [ ]:
dftot["relationType"].value_counts()

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt  
plot = dftot["relationType"].value_counts().plot.pie(y='counts', figsize=(5, 5))  
plt.show()

In [ ]:
# objekt med relatedUri borde finnas med Wikicommon objektet som sedan borde 
# peka tillbaka till K-samsök ? 
dftot.head(30)

In [ ]:
dftot.tail(30)

In [ ]:
isCommon = dftot[dftot['relatedUri'].str.contains("commons.wikimedia.org", na=False)]
isCommon.head()

In [ ]:
isCommon.info()

In [ ]:
isCommon.relationType.unique()

In [ ]:
isCommon["relationType"].value_counts()

In [ ]:
isCommon["userName"].value_counts() 

In [ ]:
isCommon.tail(100)

## [Dataroundtrip Wikicommon <-> Kulturarvsdata](#dataroundtrip) 
se även tankar om [Digitaltmuseum dataroundtrip](https://minancestry.blogspot.com/2020/06/draft-kulturnav.html#Dataroundtrip) och [blogpost](https://minancestry.blogspot.com/2020/06/check-status-kulturnav-raa.html)

Skall dataroundtrip fungera behöver vi koppla ihop bilderna så att Wikicommons vet om kulturarvsdata och vice versa

id 1079458  

http://ugc.kulturarvsdata.se/UGC-hub/api?method=retrieve&objectUri=all&contentId=1079458&scope=single  
 
``` 
<response>
 <apiVersion>1.0</apiVersion>
 <relations>
  <relationType>isVisualizedBy</relationType>
  <relatedUri>https://commons.wikimedia.org/wiki/File:Skeppshamns_kapell,_Tyndero_2006-02-27.jpg</relatedUri>
  <tagValue/>
  <coordinateValue/>
  <commentValue/>
  <imageUrl/>
  <id>1079458</id>
  <uri>http://kulturarvsdata.se/raa/kmb/16000200045362</uri>
  <userName>blajo</userName>
  <applicationName>Kringla</applicationName>
  <createDate>2013-09-14</createDate>
  <updateDate/>
 </relations>
</response> 
```
-->   [File:Skeppshamns_kapell,_Tyndero_2006-02-27.jpg](https://commons.wikimedia.org/wiki/File:Skeppshamns_kapell,_Tyndero_2006-02-27.jpg)  
1. används i Wikidata [Q30313619](https://www.wikidata.org/wiki/Q30313619)
1. Wikicommon [M723463](https://commons.wikimedia.org/wiki/Special:EntityData/M723463.json) 
```
{
   entities: {
   M723463: {
   pageid: 723463,
   ns: 6,
   title: "File:Skeppshamns kapell, Tyndero 2006-02-27.jpg",
   lastrevid: 427376612,
   modified: "2020-06-18T20:24:28Z",
   type: "mediainfo",
   id: "M723463",
   labels: { },
   descriptions: { },
   statements: {
      P180: [
          {
             mainsnak: {
                snaktype: "value",
                property: "P180",
                datavalue: {
                   value: {
                      entity-type: "item",
                      numeric-id: 30313619,
                      id: "Q30313619"
                   },
                   type: "wikibase-entityid"
                }
              },
              type: "statement",
              id: "M723463$C3F19032-D57A-4208-B967-0F4C0C8D9E3D",
              rank: "normal"
           }
        ]
     }
    }
  }
}
``` 

### Wikicommons bilden bör peka tillbaka till K-samsök URI  
K-samsök URI = [Wikidata Property 1260](https://www.wikidata.org/wiki/Property:P1260) 

==> **Ny Wikicommon** [M723463](https://commons.wikimedia.org/wiki/Special:EntityData/M723463.json) med länk ** [raa/kmb/16000200045362](http://kulturarvsdata.se/raa/kmb/16000200045362) ** 

```  
{
   entities: {
   M723463: {
   pageid: 723463,
   ns: 6,
   title: "File:Skeppshamns kapell, Tyndero 2006-02-27.jpg",
   lastrevid: 427376612,
   modified: "2020-06-18T20:24:28Z",
   type: "mediainfo",
   id: "M723463",
   labels: { },
   descriptions: { },
   statements: {
      P180: [
          {
             mainsnak: {
                snaktype: "value",
                property: "P180",
                datavalue: {
                   value: {
                      entity-type: "item",
                      numeric-id: 30313619,
                      id: "Q30313619"
                   },
                   type: "wikibase-entityid"
                }
              },
              type: "statement",
              id: "M723463$C3F19032-D57A-4208-B967-0F4C0C8D9E3D",
              rank: "normal"
           }
       ]. 
       P1260: [
          {
             mainsnak: {
               snaktype: "value",
               property: "P1260",
               datavalue: {        
                   value: "raa/kmb/16000200045362",
                   type: "string"
                 }
              },
              type: "statement",
              id: "M723463$5369f899-4667-be9a-f9ba-40d8b90dabcc",
              rank: "normal"
              }
         ]
     }
    }
  }
}
``` 



## Listar unika användare som bidragit


In [ ]:
print(dftot.userName.unique())

In [ ]:
usersActivity = dftot["userName"].value_counts() 
for i,v in dftot["userName"].value_counts().items():
    print(i,"\t",v)

In [ ]:
percentage = dftot["userName"].value_counts(normalize=True) 


In [ ]:
# Top 5 med procent andel --> 96 % gissar jag gjorts med program
percentage[:5]

In [ ]:
#Visa piechart 
plot = percentage.plot.pie(y='counts', figsize=(5, 5)) 
plt.show()

In [ ]:
# Visa top 15 
plot2 = percentage[:15].plot.bar(y='counts', figsize=(30, 5))  
plt.show()

In [ ]:
# Visa plats 6-40 
plot2 = percentage[5:40].plot.bar(y='counts', figsize=(25, 5)) 
plt.show()

In [ ]:

users = dftot["userName"].value_counts()

users.plot( kind = 'bar') 
plt.show()

In [ ]:
dftot["createDate"].value_counts()

In [ ]:
dftot["createDate"].value_counts().sort_index(ascending=False)

In [ ]:
# UGC äldsta värde är 2012-02-02 (samma år som WD skapades) och 
# mer än hälften av posterna skapades under en dag 2020-06-17 igår...?!?!?
dftot["createDate"].value_counts().sort_index()

In [ ]:
dfCreateDate = dftot["createDate"].value_counts().sort_index()
plotCreateDate = dfCreateDate.plot.pie(y='counts', figsize=(5, 5))  
plt.show()

In [ ]:
#Plot was hase been done the 100 latest  
plotCreateDate1 =   dfCreateDate.sort_index(ascending=False)[:100].plot.pie(y='counts', figsize=(10, 10))  
plt.show()

In [ ]:
plotCreateDate1 =   dfCreateDate.sort_index(ascending=False)[:100].plot.bar(y='counts', figsize=(5, 5))  
plt.show()

In [ ]:
dfCreateDate.sort_index(level=0) 

In [ ]:
dfCreateDate.sort_index(level=0)

Plot piechart user

check links Europeana and if we have Linkeddata to agents ....

In [ ]:
europeana = dftot['relatedUri'].str.contains("data.europeana.eu") 
europeana.describe()

In [ ]:
# 75 procent är länkar till Europeana dvs. troligen att samma objekt flyttats till Europeana 
# 
plotEuropeana = europeana.value_counts().plot.pie(y='counts', figsize=(5, 5))  

## Koll om det finns kopplingar länkad data dvs agent  
1966045 kopplingar verkar vara mellan objekt i kulturarvsdata till data.europeana.eu/item/  
Koppling Länkad data hittad dvs. samma som [Europeana Entity](https://www.wikidata.org/wiki/Property:P7704) samma som [Europeana Entity API](https://pro.europeana.eu/page/entity) verkar vara 46 stycken jmf Wikidata Europeana Entity > [160 000](https://www.wikidata.org/wiki/Property:P7704). Gissar att man pekar på uppladdade objekt... 

Dvs. det är i princip ingen semantik som tillförs utan mer info om att en bild flyttats. Europeana i sig är väldigt svagt semantiskt utan verkar bara sitta och gissa se [the sad story of Carl Larsson](https://minancestry.blogspot.com/2020/03/carl-larsson-who-is-that-sadly.html)

In [ ]:
europeanaagent = dftot['relatedUri'].str.contains("agent","data.europeana.eu") 
europeanaagent2 = europeanaagent[europeanaagent == True] 
europeanaagent2.describe()

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
pd.options.display.max_rows

In [ ]:
#Save session 
#import dill 
#dill.dump_session('ugc_env.db') 
# dill.load_session('ugc_env.db')

## Svensk runbibliografi

In [ ]:
Svenskrunbibliografi = dftot[dftot['applicationName'].str.contains("Svensk runbibliografi",na=False)] 


In [ ]:
Svenskrunbibliografi.relationType.unique()

In [ ]:
Svenskrunbibliografi.info()

In [ ]:
Svenskrunbibliografi.userName.unique()

In [ ]:
Svenskrunbibliografi.createDate.unique()

In [ ]:
Svenskrunbibliografi.comment.unique()

In [ ]:
Svenskrunbibliografi.relatedUri.head(10)

In [ ]:
Svenskrunbibliografi.head(20)

Still using the old system not LIBRISXL and LIBRISXL seems not link back RAÄ....
* http://libris.kb.se/bib/2839747 correct is   
  * https://libris.kb.se/katalogisering/wd6wddh75pcpmxv ==> 
    * https://libris.kb.se/wd6wddh75pcpmxv/data.jsonld 

In [ ]:
# get number of  libris items we reference  
libris = set() 
librisurl = "http://libris.kb.se/resource/bib/"
for index, row in Svenskrunbibliografi.iterrows():
    librisref = row['relatedUri'].split(".se/")[1].split("bib/")[1]
    libris.add(librisref)
    url = librisurl + librisref 
    # Looks like we get status 500 internal error from LIBRIS
    #try: 
        #print (url,librisref)
        #r = http.request('GET', url) 
        #print(r.headers) 
    #except : 
        #print ("\tError")


print ("Number of books linking: ",len(libris))
    

In [ ]:
carwashCreated = dftot[dftot['userName'].str.contains("carwash",na=False)] 


In [ ]:
carwashCreated.head(10)

In [ ]:
carwashCreated.info()

In [ ]:
carwashCreatedDate = carwashCreated["createDate"].value_counts().sort_index()
plotcarwashCreateDate = carwashCreatedDate.plot.pie(y='counts', figsize=(5, 5))  
plt.show()

In [ ]:
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
sns.heatmap(dftot.isnull(), 
            yticklabels=False, 
            cbar=False, 
            cmap='viridis')

In [ ]:
#Show empty fields --> no coordinate values 
sns.heatmap(carwashCreated.isnull(), 
            yticklabels=False, 
            cbar=False, 
            cmap='viridis')